## Select a MSQ Tide file

In [1]:
using CSV
using Dates, DataFrames, DSP
using LaTeXStrings
using NativeFileDialog
using Plots, Printf
using Statistics
using Tk

################################################
################################################
################################################
##           START OF MAIN PROGRAM
################################################
################################################
################################################

# Widen screen for better viewing
display("text/html", "<style>.container { width:100% !important; }</style>")

# Pick directory containing MSQ .csv files
msq_directory = pick_folder()

# build list of all msq files in selected directory
msq_files = filter(x->occursin("_10min",x), readdir(msq_directory));
##msq_files = filter(x->occursin("_hourly",x), readdir(msq_directory));
msq_files = msq_files[findall(x->endswith(uppercase(x), ".CSV"), msq_files)]

# Check whether any msq files exist in selected directory. If not, EXIT!
if length(msq_files) == 0
    println("No msq files found in "*msq_directory)
    exit;
else
    println(string(length(msq_files)) * " MSQ tide files found")
end

w = Toplevel("Select Date", 235, 600)
tcl("pack", "propagate", w, false)
f = Frame(w)
pack(f, expand=true, fill="both")

f1 = Frame(f)
lb = Treeview(f1, msq_files)
scrollbars_add(f1, lb)
pack(f1,  expand=true, fill="both")

tcl("ttk::style", "configure", "TButton", foreground="blue", font="arial 16 bold")
b = Button(f, "Ok")
pack(b)

bind(b, "command") do path
    
    global file_choice = get_value(lb);
    
    # Select a msq file
    global infil = msq_directory * "\\" * file_choice[1]
    
    global title = file_choice[1]
    
    println("Selected ",infil)

###    global msq_df = DataFrame(CSV.File(infil; header=false, skipto=42, delim=", ", ignorerepeated=true));
###    msq_df.Date = DateTime.(DateTime.(strip.(msq_df.Column1),"dd/mm/yyyy HH:MM"))      
###    rename!(msq_df,[:Column2,:Column3] .=> [:Ind, :Reading]);
###    select!(msq_df, [:Date, :Ind, :Reading], Not([:Column1]));
    
    global msq_df = DataFrame(CSV.File(infil; header=false, skipto=42, delim=", ", ignorerepeated=true))
    msq_df.Column1 = DateTime.(msq_df.Column1.*chop.(msq_df.Column2,tail=1), dateformat"dd/mm/yyyy HH:MM")
    rename!(msq_df,[:Column1, :Column3, :Column4] .=> [:Date, :Ind, :Reading]);
    select!(msq_df, [:Date, :Ind, :Reading], Not([:Column2]));

    msq_df.Reading = parse.(Float64,msq_df.Reading)
    
    # Convert -9999 values to Nans for plotting
    msq_df.Reading[findall(abs.(msq_df.Reading).>9)] .= NaN
    
    msq_df.time_diff = [missing; diff(msq_df.Date)]
    
    global first_date = first(msq_df.Date)
    global last_date = last(msq_df.Date)
    
    year_diff = (Year(last_date) |> Dates.value) - (Year(first_date) |> Dates.value)
    
    # set X-axis scale based on amount of data
    if (year_diff == 0)
        mth_val = 1
    elseif (year_diff == 1)
        mth_val = 2
    elseif (year_diff == 2)
        mth_val = 3
    else
        mth_val = 6
    end
    
    # display plots to screen
    tm_tick = range(first_date,last_date,step=Year(2))
    ticks = Dates.format.(tm_tick,"YY")
    
    tides_plot = plot(msq_df.Date,msq_df.Reading, size = (1400, 600), label="",
##        marker = :circle, markersize = 1,
        xlims=(first_date,last_date), xticks=(tm_tick,ticks), xtickfontsize=7,ytickfontsize=8, 
        framestyle = :box, legend=:bottomleft, title=title, 
        margin = 1Plots.mm, grid=true, gridlinewidth=0.5, gridstyle=:dot, gridalpha=1)
    
    display(tides_plot)
    
    start_date = first_date
    println(title,"\n")
    println("Mth Year  Gaps     Good     Minimum     Maximum     Mean      Range    St Devn")

    total_gaps = 0
    total_good = 0

    yearly_mean = 0
    no_of_months = 0
    
    global yearly_averages_df = DataFrame([[],[]], ["Year", "Mean_WL"])
    
    global monthly_averages_df = DataFrame(Any[Int[], Int[], Int[], Int[], Float64[], Float64[], Float64[], Float64[], Float64[]], ["Month", "Year", "Gaps", "Good", "Minimum", "Maximum", "Mean", "Range", "St Devn"])
    
    while start_date <= last_date
        
        # select a month of data and store month of data in temporary df
        end_date = start_date + Dates.Month(1)
        month_df = msq_df[findall(start_date .<= msq_df.Date .< end_date),:]

        # determine total number of 10-minute records possible for the month
        total_possible = trunc(Int,(end_date - start_date) / Millisecond(1) * (1 / 3.6e+6))
        total_actual = nrow(month_df)
        
        # determine the number of gaps
        gaps = total_possible - total_actual

        # get totals of gaps and good data
        total_gaps = total_gaps + gaps
        total_good = total_good + total_actual

        # get monthly values
        month = Month(start_date) |> Dates.value
        year = Year(start_date) |> Dates.value
        
        if nrow(month_df) > 1
            monthly_min = minimum(month_df.Reading)
            monthly_max = maximum(month_df.Reading)
            monthly_mean = mean(month_df.Reading)
            monthly_std = std(month_df.Reading)
            monthly_range = monthly_max - monthly_min

        else
            monthly_min = 0
            monthly_max = 0
            monthly_mean = 0
            monthly_std = 0
            monthly_range = 0
        end

        @printf("%3i %4i %5i %8i %10.3f %11.3f %10.3f %9.3f %9.3f\n", month, year, gaps, total_actual, monthly_min, monthly_max, monthly_mean, monthly_range, monthly_std)
        push!(monthly_averages_df,[month, year, gaps, total_actual, monthly_min, monthly_max, monthly_mean, monthly_range, monthly_std])
        
        # add monthly mean to yearly total, and increment number of months used
        yearly_mean = yearly_mean + monthly_mean
        no_of_months = no_of_months + 1

        # print annual averages for current year of data
        if (Year(end_date) |> Dates.value) != (Year(start_date) |> Dates.value)
            
            year_valu = Year(start_date) |> Dates.value
            year_total = mean(msq_df[findall(Year.(msq_df.Date) .|> Dates.value .== year),:].Reading)
            # add yearly average to df
            push!(yearly_averages_df,[year_valu,year_total])
            
            println("=============================================================================")
            @printf("%s %4i %s %5.3f\n","                         Yearly average for",(year_valu),"= ", year_total)
            @printf("%s%5.3f%s\n","                           From Monthly Averages = (",yearly_mean/no_of_months,")")
            println("=============================================================================\n")
            
            # reset counters
            yearly_mean = 0
            no_of_months = 0
        end

        # move to start of next month
        start_date = end_date

    end

    println("Total number of Gaps = ",total_gaps,"; Total number of Good values = ", total_good,"\n")
    println(title)
    println("    Year       Mean")
    for i in 1:nrow(yearly_averages_df)
        @printf("%8i %10.3f\n",yearly_averages_df.Year[i],yearly_averages_df.Mean_WL[i])
    end
    flush(stdout)
    
    CSV.write(".\\" * splitext(last(splitdir(infil)))[1] * "_monthly_averages.CSV",monthly_averages_df)
end

48 MSQ tide files found


""

Selected G:\SLR\Daily_Hi_Lo\R060008A  HAY POINT STORM SURGE_01.01.10-31.12.20_10min_daily_hi_lo_mean.CSV
error during Tk callback: 


ERROR: ArgumentError: column name "Column2" not found in the data frame; existing most similar names are: "Column1"
Stacktrace:
 [1] lookupname
   @ C:\Users\Jim\.julia\packages\DataFrames\58MUJ\src\other\index.jl:416 [inlined]
 [2] getindex
   @ C:\Users\Jim\.julia\packages\DataFrames\58MUJ\src\other\index.jl:422 [inlined]
 [3] getindex(df::DataFrame, #unused#::typeof(!), col_ind::Symbol)
   @ DataFrames C:\Users\Jim\.julia\packages\DataFrames\58MUJ\src\dataframe\dataframe.jl:557
 [4] getproperty(df::DataFrame, col_ind::Symbol)
   @ DataFrames C:\Users\Jim\.julia\packages\DataFrames\58MUJ\src\abstractdataframe\abstractdataframe.jl:431
 [5] (::var"#7#8")(path::String)
   @ Main .\In[1]:67
 [6] jl_tcl_callback(fptr::Ptr{Nothing}, interp::Ptr{Nothing}, argc::Int32, argv::Ptr{Ptr{UInt8}})
   @ Tk C:\Users\Jim\.julia\packages\Tk\c8ZUf\src\tkwidget.jl:142
 [7] tcl_doevent (repeats 2 times)
   @ C:\Users\Jim\.julia\packages\Tk\c8ZUf\src\tkwidget.jl:20 [inlined]
 [8] macro expansion
   @ .\as

In [ ]:
msq_df = DataFrame(CSV.File(infil; header=false, skipto=42, delim=", ", ignorerepeated=true))
msq_df.Column1 = DateTime.(msq_df.Column1.*chop.(msq_df.Column2,tail=1), dateformat"dd/mm/yyyy HH:MM")
rename!(msq_df,[:Column1, :Column3, :Column4] .=> [:Date, :Ind, :Reading]);
select!(msq_df, [:Date, :Ind, :Reading], Not([:Column2]));

In [ ]:
aa

In [ ]:
CSV.write(".\\" * splitext(last(splitdir(infil)))[1] * "_monthly_averages.CSV",monthly_averages_df)

In [ ]:
trunc(Int,Month(start_date) |> Dates.value)

In [ ]:
using Dates

print(Dates.monthname(2))

In [ ]:
msq_df